In [34]:
# GET LIST FROM BDR. CRAFT A COLD OUTREACH EMAIL TO CONTACT 1 OF EACH COMPANY BASED ON COMPANY DATA (SHOULD BE IN CHATGPT) AND PCIO DATA
# DATA SHOULD ALREADY BE IN HUBSPOT AT THIS POINT. 

#GIVE THE CLIENT SOMETHING. EITHER SHOW YOU KNOW ABOUT THEIR BUSINESS, SHOW YOU KNOW THEIR PAIN POINTS, SHOW YOU KNOW SOMETHING. KEEP EMAIL SHORT SO IT DOESNT LOOK LIKE AI.

# THE TRAINING AND GENERATING A NOT SHITTY COLD EMAIL IS THE PRIMARY OBJECTIVE HERE. 



#All of the dependencies are listed in the .toml file. Ensure those are going with Poetry
#ensure your .env file is in the right place 

#make sure you're using poetry shell to get all of the dependencies going in your folder

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

print(os.getcwd())
dotenv_path = find_dotenv()
print(f"Loading .env file from: {dotenv_path}")
_ = load_dotenv(dotenv_path)


from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

creativeLlmModel = OpenAI(temperature=0.9)
chatModel = ChatOpenAI(model="gpt-4o-mini")
llmModel = OpenAI()

/Users/kevincorstorphine/Desktop/Agentic_Start
Loading .env file from: /Users/kevincorstorphine/Desktop/Agentic_Start/.env


In [35]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("model_background_personality.txt")
scrape_data1 = TextLoader("scraped_pages/1example_com_combined.txt")

loaded_data = loader.load()
scrape_data1 = scrape_data1.load()


In [36]:
#If you need smaller chunks, you can enforce a maximum chunk size
#to ensure that the splitter splits. It looks for logical places to split
#and the .txt scrapes are jumbled and don't have logical breaks.

# It may make sense to 


#Splitters to call small sections of relevant data, but it is also limiting in how to craft a creative response AFAIK

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [37]:
load = text_splitter.create_documents([loaded_data[0].page_content])
scrape1 = text_splitter.create_documents([scrape_data1[0].page_content])

Created a chunk of size 4565, which is longer than the specified 1000
Created a chunk of size 4565, which is longer than the specified 1000
Created a chunk of size 1711, which is longer than the specified 1000
Created a chunk of size 3141, which is longer than the specified 1000
Created a chunk of size 3098, which is longer than the specified 1000
Created a chunk of size 1360, which is longer than the specified 1000
Created a chunk of size 2155, which is longer than the specified 1000
Created a chunk of size 1037, which is longer than the specified 1000
Created a chunk of size 1738, which is longer than the specified 1000
Created a chunk of size 27868, which is longer than the specified 1000
Created a chunk of size 1730, which is longer than the specified 1000
Created a chunk of size 29895, which is longer than the specified 1000


In [38]:
len(load)
len(scrape1)


58

In [ ]:
# TO DO BETTER PROMPTS READ THE E-BOOK ON PROMPTING


#I may need to use vector databases here, or maybe not. not totally sure yet.

In [41]:

#Loaded_data is the personality I gave the model in the model_background_personality.txt file. I told it to be a pirate. In the live version
#change this to the background about API3 and the product. 

#it is a pirate doing cold outreach selling bananas. 



from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {kind_of_entity}."),
        ("human", "Hello, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}{scrape_detail}"),
    ]
)

messages = chat_template.format_messages(
    kind_of_entity=str(load),
   # topic="pirating",
    user_input= "Please write an outbound cold email about the problems they may be facing without a reliable helpdesk IT support service",
    scrape_detail=str(scrape1),
)

response = chatModel.invoke(messages)

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

In [42]:
print(response.content) 

Subject: Is Your IT Support Holding You Back?

Hi [Recipient's Name],

I hope this message finds you well. I wanted to reach out because many businesses today face challenges that can significantly impact their productivity and bottom line, especially when it comes to IT support.

Are you experiencing any of the following issues?

- Frequent downtime that disrupts operations and frustrates your team?
- Slow response times when technical problems arise, leading to lost productivity?
- Difficulty in managing IT resources effectively, causing delays in critical projects?
- Security concerns that keep you up at night, knowing your systems may not be adequately protected?

At PCIO, we understand that a reliable helpdesk IT support service can be a game-changer. Our dedicated team focuses on providing solutions tailored to your needs, helping you overcome these challenges and ensuring your business runs smoothly.

If any of these problems resonate with you, I would love to have a conversatio

In [5]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate


#examples are a way to train the model to react exactly how you want it to.
#in the class example, this was is way to get it to speak Spanish
#FewshotChatprompttemplate is just chatPromptTemplate with examples
#Use this further modeling to refine the pitch, say waht works, what doesn't, etc.

#train AI on how you want an outbound email to look. These are just two examples of how to train it:
examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " you are only targeting Spanish speaking communities and are translating to spanish"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

#a chain defines a sequence of order in which to run


chain = final_prompt | chatModel

In [6]:
print(chain.invoke({"input": "please do a sales pitch to someone based on the product you sell"}).content)

¡Claro! Imaginemos que estoy vendiendo un purificador de aire.

---

¡Hola! ¿Te has dado cuenta de cuánto tiempo pasamos en interiores? La calidad del aire que respiramos en casa o en la oficina puede afectar nuestra salud y bienestar. Te presento nuestro purificador de aire de última generación.

Este purificador no solo elimina el 99.9% de los alérgenos y contaminantes del aire, como el polvo, el polen y el moho, sino que también cuenta con un sistema de filtración HEPA que garantiza un ambiente más saludable para ti y tu familia. 

Además, su diseño elegante se adapta a cualquier espacio, y su funcionamiento es tan silencioso que apenas notarás que está trabajando. También cuenta con un temporizador y modos de funcionamiento que puedes ajustar según tus necesidades.

Imagina respirar aire fresco y limpio todos los días. ¡Haz la inversión en tu salud y bienestar hoy! Con cada compra, te ofrecemos un 20% de descuento durante este mes. No dejes pasar esta oportunidad. 

¿Qué dices? ¿Te

In [ ]:
#READ PROMPTING BOOK AND DIAL THIS ALL IN

In [21]:



# FOR ADDITIONAL JSON PARSING STUFF< CHECK OUT THE PYDANTIC LIBRARY


# from langchain_core.prompts import PromptTemplate
# from langchain.output_parsers.json import SimpleJsonOutputParser

# json_prompt = PromptTemplate.from_template(
#     "Return a JSON object with an `answer` key that answers the following question: {question}"
# )

# json_parser = SimpleJsonOutputParser()

# json_chain = json_prompt | llmModel | json_parser

In [24]:
# json_parser.get_format_instructions()

In [25]:
# json_chain.invoke({"question": "What is the biggest country?"})

SyntaxError: unterminated string literal (detected at line 1) (3837994639.py, line 1)

In [34]:
#MAKE SOME GRAPHIC BASED ON WHAT THEY'RE LIKELY SPENDING, CUSTOMIZE IT, AND SEND THEM A NEW ONE WITH HOW PCIO SOLVES THEIR PROBLEM

In [2]:
# For Mom, research Klayvio